In [1]:
import pandas as pd
import re

# 1. VERİ YÜKLEME
# ARTIK TAM YOL KULLANIYORUZ, GÖRELİ YOL SORUNUNU AŞTIK.
# r'' kullanmak, Windows yolundaki \ işaretlerini doğru okumasını sağlar.
VERI_YOLU = r"C:\Users\ozlem\OneDrive\Belgeler\GitHub\-twitter-sentiment-analizi\data\test.csv" 

try:
    df = pd.read_csv(VERI_YOLU)
    print("✅ Veri başarıyla yüklendi.")
except FileNotFoundError:
    print(f"HATA: Dosya '{VERI_YOLU}' yolunda hala bulunamıyor. Lütfen yolu manuel kontrol edin.")
    raise # Hata durumunda kodu tamamen durdururuz.

# 2. TEMEL TEMİZLİK VE FİLTRELEME
# Sadece text (tweet metni) ve label (etiket) sütunlarını alıyoruz.
df = df[['text', 'label']] 

# 'Notr' etiketlerini kaldırıyoruz. Sadece 'Positive' ve 'Negative' kalacak.
df = df[df['label'] != 'Notr'].reset_index(drop=True)
print(f"\n✅ 'Notr' etiketleri kaldırıldı. Kalan veri satır sayısı: {len(df)}")


# 3. METİN TEMİZLİK FONKSİYONU
def temel_metin_temizligi(text):
    text = str(text)
    # URL'leri, kullanıcı adlarını, noktalama ve sayıları temizler
    text = re.sub(r'http\S+|www\S+|https\S+|@\w+|[^\w\s]|\d+', '', text, flags=re.MULTILINE)
    # Birden fazla boşluğu tek boşluğa indirme ve küçük harfe çevirme
    text = re.sub(r'\s+', ' ', text).strip().lower()
    return text

# Temizleme işlemini uygulama
df['text_temiz'] = df['text'].apply(temel_metin_temizligi)

print("\n✅ Temel metin temizliği yapıldı.")
print("\nİşlenmiş Veri Kontrolü (İlk 5 Satır):")
print(df[['text', 'text_temiz', 'label']].head())


✅ Veri başarıyla yüklendi.

✅ 'Notr' etiketleri kaldırıldı. Kalan veri satır sayısı: 31873

✅ Temel metin temizliği yapıldı.

İşlenmiş Veri Kontrolü (İlk 5 Satır):
                                                text  \
0   ısrarla korkutmayı başarıyor. sanki korku çok...   
1  i phone 5 ten sonra gene 4'' ekranı tercih ett...   
2  arkadaşımın ofisinde gördüm,  tavsiyesi üzerin...   
3   cok sıkıcı bir film birkaç oyuncunun iyi perf...   
4  ürün çok kısa bir sürede geldi ve çok kullanış...   

                                          text_temiz     label  
0  ısrarla korkutmayı başarıyor sanki korku çok u...  Positive  
1  i phone ten sonra gene ekranı tercih ettim tel...  Positive  
2  arkadaşımın ofisinde gördüm tavsiyesi üzerine ...  Positive  
3  cok sıkıcı bir film birkaç oyuncunun iyi perfo...  Negative  
4  ürün çok kısa bir sürede geldi ve çok kullanış...  Positive  


In [4]:
import nltk
# Türkçe tokenization için gereken kaynak
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\ozlem\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt_tab.zip.


True

In [7]:

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

# NLTK kütüphanesini kullanmak için gerekli Türkçe dosyasını indirme (ilk çalıştırmada gereklidir)
try:
    # Bu adımı daha önce yapmıştık, ama emin olmak için burada kalsın.
    nltk.download('punkt')
except:
    print("NLTK 'punkt' zaten yüklü.")

# --- 1. Türkçe Stopwords Listesi ---
turkce_stopwords = set([
    "a", "acaba", "ama", "aslında", "az", "bazı", "belki", "biri", "birkaç", "birşey", "biz", "böyle", 
    "bunu", "çünkü", "çok", "çoğu", "de", "da", "daha", "diye", "eğer", "fakat", "falan", "filan", 
    "gibi", "hâlâ", "hangi", "hemen", "her", "herkes", "hiç", "için", "ile", "ise", "ya", "ve", 
    "kim", "mı", "mı", "mu", "mü", "nasıl", "ne", "neden", "nedense", "nerde", "nereye", "niye", 
    "o", "oysa", "oysaki", "öbürü", "ön", "şayet", "şey", "şu", "tüm", "tek", "üzere", "yoksa",
    "bu", "şu", "ki", "mi", "mı", "mu", "mü", "ne", "niye", "hangi", "kime", "kimden"
])

# --- 2. Stopwords Kaldırma ve Tokenization Fonksiyonu ---
def stopword_ve_tokenize(text):
    tokens = word_tokenize(text, language='turkish')
    filtered_tokens = [word for word in tokens if word not in turkce_stopwords and len(word) > 1]
    return " ".join(filtered_tokens) 

# Uygulama
df['text_temiz'] = df['text_temiz'].apply(stopword_ve_tokenize)

print("✅ Stopwords ve Tokenization tamamlandı.")
print("\nİşlenmiş Veri Kontrolü:")
print(df[['text', 'text_temiz']].head())



[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\ozlem\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


✅ Stopwords ve Tokenization tamamlandı.

İşlenmiş Veri Kontrolü:
                                                text  \
0   ısrarla korkutmayı başarıyor. sanki korku çok...   
1  i phone 5 ten sonra gene 4'' ekranı tercih ett...   
2  arkadaşımın ofisinde gördüm,  tavsiyesi üzerin...   
3   cok sıkıcı bir film birkaç oyuncunun iyi perf...   
4  ürün çok kısa bir sürede geldi ve çok kullanış...   

                                          text_temiz  
0  ısrarla korkutmayı başarıyor sanki korku uzun ...  
1  phone ten sonra gene ekranı tercih ettim telef...  
2  arkadaşımın ofisinde gördüm tavsiyesi üzerine ...  
3  cok sıkıcı bir film oyuncunun iyi performansı ...  
4  ürün kısa bir sürede geldi kullanışlı arabamız...  


In [8]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

# --- 1. Etiketleri Sayısallaştırma (Negative=0, Positive=1) ---
le = LabelEncoder()
# Etiketleri dönüştürme: Negative -> 0, Positive -> 1
df['label_sayisal'] = le.fit_transform(df['label']) 
print("✅ Etiketler Sayısallaştırıldı. (0: Negative, 1: Positive)")

# --- 2. Veriyi Bağımlı ve Bağımsız Değişkenlere Ayırma ---
# X: Temizlenmiş Metin (bağımsız değişken)
# y: Sayısallaştırılmış Etiket (bağımlı değişken)
X = df['text_temiz']
y = df['label_sayisal']

# --- 3. Eğitim ve Test Setlerine Ayırma (Model Eğitimi için) ---
# Verinin %80'i eğitim, %20'si test için kullanılacak.
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

print(f"\n✅ Veri, eğitim ve test setlerine ayrıldı.")
print(f"Eğitim Seti Büyüklüğü: {len(X_train)} satır")
print(f"Test Seti Büyüklüğü: {len(X_test)} satır")

✅ Etiketler Sayısallaştırıldı. (0: Negative, 1: Positive)

✅ Veri, eğitim ve test setlerine ayrıldı.
Eğitim Seti Büyüklüğü: 25498 satır
Test Seti Büyüklüğü: 6375 satır


In [10]:
from sklearn.feature_extraction.text import TfidfVectorizer

# TF-IDF Vektörleyici Oluşturma
# max_features: En sık kullanılan 5000 kelimeyi alıyoruz.
# ngram_range: Tekli kelimelerin yanı sıra ikili kelime gruplarını da dahil ediyoruz.
vectorizer = TfidfVectorizer(max_features=5000, ngram_range=(1, 2))

# Eğitim verisi üzerinde eğitme ve DÖNÜŞTÜRME
X_train_vec = vectorizer.fit_transform(X_train)

# Test verisini dönüştürme (Sadece transform, fit değil!)
X_test_vec = vectorizer.transform(X_test)

print("✅ TF-IDF Vektörleştirme tamamlandı.")
print(f"Eğitim Matrisi Şekli (Satır, Özellik/Kelime Sayısı): {X_train_vec.shape}")

✅ TF-IDF Vektörleştirme tamamlandı.
Eğitim Matrisi Şekli (Satır, Özellik/Kelime Sayısı): (25498, 5000)
